In [1]:
from datetime import datetime, date
import os

import pandas as pd
import pyspark 
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import pyspark.pandas as ps

spark = SparkSession.builder.appName("Practice").getOrCreate()

22/06/29 12:35:45 WARN Utils: Your hostname, jacob-BigOtisLinux resolves to a loopback address: 127.0.1.1; using 192.168.50.142 instead (on interface enp6s0)
22/06/29 12:35:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/29 12:35:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
customers_last_yr = spark.createDataFrame(
    [(1, 'Sam'),
     (2, 'Jacob'),
     (3, 'Andrew')
    ],
    ['customer_id', 'name']
)

customers_this_yr = spark.createDataFrame(
    [(1, 'Sam'),
     (4, 'Dan'),
     (5, 'Roger')
    ],
    ['customer_id', 'name']
)

new_customers = customers_this_yr.join(customers_last_yr, on = ['customer_id'], how = 'leftanti')

new_customers.show()

+-----------+-----+
|customer_id| name|
+-----------+-----+
|          4|  Dan|
|          5|Roger|
+-----------+-----+



In [7]:
new_customers.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [customer_id#26L], [customer_id#22L], LeftAnti
   :- Sort [customer_id#26L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(customer_id#26L, 200), ENSURE_REQUIREMENTS, [id=#136]
   :     +- Scan ExistingRDD[customer_id#26L,name#27]
   +- Sort [customer_id#22L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(customer_id#22L, 200), ENSURE_REQUIREMENTS, [id=#137]
         +- Project [customer_id#22L]
            +- Filter isnotnull(customer_id#22L)
               +- Scan ExistingRDD[customer_id#22L,name#23]




In [15]:
# coalesce is used to limit the number of files you write when doing write csv / write parquet options
# havin 1000 small files for a 50mb dataset you're saving is dumb.
# coalesce changes the number of partitions in your dataset.
# dont have to load it into pandas and then write pd.write_csv(), just do coalsece(1)

df = spark.read.csv('nba_tweets.csv', header = True)
df.write.csv(f"tweets_test_{datetime.now().date()}", header = True) # stores 3 files
df.coalesce(1).write.csv(f"tweets_test_coalesce_{datetime.now().date()}", header = True) # stores 1 file

# new_customers.coalesce(1).write.csv(f"new_customers_{datetime.now().date()}", header = True)

In [20]:
games = spark.createDataFrame(
    [('GSW vs DAL', '2022-01-01')],
    ['game_event', 'game_date_str']
)

# these 2 epxressions do the same thing.
games = games.withColumn('game_date', F.col('game_date_str').cast('date'))
games = games.selectExpr("*", "CAST(game_date_str as DATE) as game_date_expr")

games.show()
games.printSchema()

+----------+-------------+----------+--------------+
|game_event|game_date_str| game_date|game_date_expr|
+----------+-------------+----------+--------------+
|GSW vs DAL|   2022-01-01|2022-01-01|    2022-01-01|
+----------+-------------+----------+--------------+

root
 |-- game_event: string (nullable = true)
 |-- game_date_str: string (nullable = true)
 |-- game_date: date (nullable = true)
 |-- game_date_expr: date (nullable = true)

